### 单独分析给药->LOC或停药->ROC，看看哪些feature与意识水平的逐步下降或逐步上升相关

In [ ]:
# 抓取给药开始和结束时间
# 0428: 停止操作（最后一个病人未记录）
# 0516～0519: 停药
# 0523～0526: 给药结束时间

In [ ]:
from datetime import datetime

import mne

from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, f1_score, r2_score

from bis_lib import *
from bis_lib_ML import feature_norm

In [ ]:
def get_drug_process(eeg_feature_dict, xls_info, key):
    label = []
    feature = []
    bis_x = np.arange(xls_info['bis_end']-xls_info['bis_start']+1)+(xls_info['bis_start'])
    eeg_feature_x = np.arange(xls_info['eeg_end_idx']-xls_info['eeg_start_idx'])+(xls_info['eeg_start_idx'])
    # 有可能因为丢包，EEG会比预想的少1min
    if eeg_feature_dict['PE'].shape[0] < eeg_feature_x.shape[0]:
        eeg_feature_x = eeg_feature_x[0:-1]
    # print(eeg_feature_x)
    if key=='bis':
        draw_x = bis_x
        draw_y = xls_info['bis']
    else:
        draw_x = eeg_feature_x
        draw_y = eeg_feature_dict[key]
    for i in range(draw_x.shape[0]):
        # if xls_info['time_inject'] <= draw_x[i] <= xls_info['time_loc']:
        #     label.append(1 - (draw_x[i] - xls_info['time_inject']) / 
        #                  (xls_info['time_loc'] - xls_info['time_inject']))
        #     feature.append(draw_y[i])
        if xls_info['time_druge'] <= draw_x[i] <= xls_info['time_roc']:
            label.append((draw_x[i] - xls_info['time_druge']) / 
                         (xls_info['time_roc'] - xls_info['time_druge']))
            feature.append(draw_y[i])
        # elif (draw_x[i] < xls_info['time_inject']) or (draw_x[i] > xls_info['time_roc']):
        #     label.append(1)
        #     feature.append(draw_y[i])
        # elif xls_info['time_loc'] < draw_x[i] < xls_info['time_druge']:
        #     label.append(0)
        #     feature.append(draw_y[i])            
    return feature, label

In [ ]:
data_info = {}
xy_learning = {}
feature_clean = True
fs = 1000
exp_dates = ['0428', '0516', '0517', '0518', '0519', '0523', '0524', '0525', '0526']

for exp_date in exp_dates:
# for exp_date in ['0523']:
    path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
    exp = get_exp(path_root, exp_date)
    eeg_start_time, eeg_end_time = get_eeg_time(path_root, exp)
    csv_path = '../csv_files/{}'.format(exp_date)
    for i in range(len(exp)):
    # for i in range(3,4):
        if exp_date=='0428' and i==3: # 没有停药标记
            continue
        if exp_date=='0523' and i==3: # 时间轴缺失
            continue
        eeg = csv2eeg(path_root, exp[i])
        print('eeg.shape', eeg.shape)
        print('duration(s):', eeg.shape[1]/fs)
        data_info['exp{}_duration'.format(i)] = eeg.shape[1]/fs
        tmp = datetime.strptime(eeg_end_time[i], '%Y-%m-%d %H:%M:%S') - datetime.strptime(eeg_start_time[i], '%Y-%m-%d %H:%M:%S')
        data_info['exp{}_pack_loss'.format(i)] = tmp.seconds - eeg.shape[1]/fs

        ch_num = eeg.shape[0]
        # for mne object
        ch_names = ['EEG{}'.format(j) for j in range(ch_num)]
        ch_types = ['eeg' for i in range(ch_num)]
        info = mne.create_info(ch_names, ch_types=ch_types, sfreq=fs)
        print(info)

        data_info['exp{}_chnum'.format(i)] = eeg.shape[0]
        data_info['exp{}_filename'.format(i)] = exp[i]

        my_raw = mne.io.RawArray(eeg, info)
        my_raw.filter(l_freq=0.5, h_freq=30)
        # draw_eeg_psd(my_raw, ch_names, exp[i])

        if exp_date in ['0523', '0524', '0525', '0526']:
            xls_info = process_xls_new(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
        else:
            xls_info = process_xls(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
            
        eeg_data = eeg_segment(my_raw[:][0], fs, seg_length=60, eeg_start_time=eeg_start_time[i])

        eeg_feature_dict = {}
        if feature_clean:
            pe = np.load('../npy_files/feature_clean/PE_{}_{}.npy'.format(exp_date, exp[i]))
            # se = np.load('../npy_files/feature_clean/SE_{}_{}.npy'.format(exp_date, exp[i]))
            lzc = np.load('../npy_files/feature_clean/LZC_{}_{}.npy'.format(exp_date, exp[i]))
            svd = np.load('../npy_files/feature_clean/SVD_{}_{}.npy'.format(exp_date, exp[i]))
        else:
            pe = np.load('../npy_files/PE_{}_{}.npy'.format(exp_date, exp[i]))
            # se = np.load('../npy_files/SE_{}_{}.npy'.format(exp_date, exp[i]))
            lzc = np.load('../npy_files/LZC_{}_{}.npy'.format(exp_date, exp[i]))
            svd = np.load('../npy_files/SVD_{}_{}.npy'.format(exp_date, exp[i]))

        eeg_feature_dict['PE'] = np.mean(pe, axis=1)
        # eeg_feature_dict['SE'] = se
        eeg_feature_dict['LZC'] = np.mean(lzc, axis=1)
        eeg_feature_dict['SVD_0'] = svd[0,:]
        eeg_feature_dict['SVD_1'] = svd[1,:]
        eeg_feature_dict['SVD_2'] = svd[2,:]
        eeg_feature_dict['SVD_3'] = svd[3,:]
        
        # draw_results(exp_date, exp[i], xls_info, eeg_feature_dict)
        for key in eeg_feature_dict.keys():
            xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_label'.format(key, exp_date, exp[i])] = get_drug_process(eeg_feature_dict, xls_info, key)
        key = 'bis'
        xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_label'.format(key, exp_date, exp[i])] = get_drug_process(eeg_feature_dict, xls_info, key)

In [ ]:
key_all = {}
key_all['BIS'] = ['bis']
key_all['MIX'] = ['PE', 'LZC', 'SVD_0', 'SVD_1', 'SVD_2', 'SVD_3']
key_all['PE'] = ['PE']
key_all['LZC'] = ['LZC']
key_all['SVD_0'] = ['SVD_0']
key_all['SVD_1'] = ['SVD_1']
key_all['SVD_2'] = ['SVD_2']
key_all['SVD_3'] = ['SVD_3']

In [ ]:
def get_model_data(exp_date_test, exp_idx_test, key_bkp):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for exp_date in exp_dates:
        path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
        exp = get_exp(path_root, exp_date)
        eeg_start_time, eeg_end_time = get_eeg_time(path_root, exp)
        csv_path = '../csv_files/{}'.format(exp_date)
        for i in range(len(exp)):
            if exp_date=='0428' and i==3:
                continue
            if exp_date=='0523' and i==3:
                continue
            for j in range(len(xy_learning['{}_{}_{}_feature'.format(key_bkp[0], exp_date, exp[i])])):
                tmp = []
                for key in key_bkp:
                    tmp.append(xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])][j])
                if exp_date==exp_date_test and i==exp_idx_test:
                    test_x.append(tmp)
                    test_y.append(xy_learning['{}_{}_{}_label'.format(key_bkp[0], exp_date, exp[i])][j])      
                else:      
                    train_x.append(tmp)
                    train_y.append(xy_learning['{}_{}_{}_label'.format(key_bkp[0], exp_date, exp[i])][j])
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    print('train_x.shape', train_x.shape)
    print('test_x.shape', test_x.shape)
    print('train_y.shape', train_y.shape)
    print('test_y.shape', test_y.shape)
    return train_x, train_y, test_x, test_y

In [ ]:
r2_mean_feature = []
r2_std_feature = []
for key in key_all.keys():
    r2_list = []
    for exp_date in exp_dates:
        path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
        exp = get_exp(path_root, exp_date)
        for i in range(len(exp)): 
            if exp_date=='0428' and i==3:
                continue
            if exp_date=='0523' and i==3:
                continue
            train_x, train_y, test_x, test_y = get_model_data(exp_date_test=exp_date, exp_idx_test=i, key_bkp=key_all[key])
            train_x, test_x = feature_norm(train_x, test_x)
            clf = SVR(kernel='rbf', C=500, gamma='auto', max_iter=1000000)
            clf.fit(train_x, train_y)
            y_hat = clf.predict(test_x)
            print("得分:", r2_score(test_y, y_hat))
            r2_list.append(r2_score(test_y, y_hat))
    print('r2:{:.3f}'.format(np.mean(np.array(r2_list))))
    r2_mean_feature.append(np.mean(np.array(r2_list)))
    r2_std_feature.append(np.std(np.array(r2_list)))

In [ ]:
# 特征归一化，留一被试法，平均r2
feature_str = ['BIS', 'MIX', 'PE', 'LZC', 'SVD_0', 'SVD_1', 'SVD_2', 'SVD_3']
# r2_raw = [0.073, -2.106, 0.176, -0.060, -0.596, -0.669, -2.222, -1.580]
r2_raw = np.array(r2_mean_feature)
r2_all = []
for i in range(len(r2_raw)):
    if r2_raw[i] < 0:
        r2_all.append(0)
    else:
        r2_all.append(r2_raw[i])

In [ ]:
plt.figure(figsize=(8, 4))
color_bkp = ['forestgreen', 'darkorange',  'firebrick',
                'limegreen', 'royalblue', 'darkgrey', 'forestgreen', 'darkblue', 'purple']
x_bar = np.arange(len(r2_all))
width = 0.4
plt.bar(x_bar, r2_all, width=width, color=color_bkp[0], edgecolor='k', label='r2')
plt.bar(x_bar, np.zeros(len(r2_all)), tick_label=feature_str)

plt.xticks(fontproperties = 'Arial', size = 14)
plt.yticks(fontproperties = 'Arial', size = 14)
plt.ylabel('R2', font={'family':'Arial', 'size':16})
plt.xlabel('Feature', font={'family':'Arial', 'size':16})

# plt.legend(loc=(1.05, 0.3), ncol=1, edgecolor='k', prop={'family':'Arial', 'size':14})
plt.tight_layout()
# plt.savefig('figs/drug_process.png', dpi=300, bbox_inches='tight')
plt.show()